In [1]:
!pip -q install -U "openai>=1.54.0" datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.8/810.8 kB 17.6 MB/s eta 0:00:00


In [2]:
# # Configure OpenAI-compatible Gemini API

import os
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"  
REASONING_EFFORT = "low"            

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)

print("Client ready with OpenAI-compatible Gemini endpoint.")


Client ready with OpenAI-compatible Gemini endpoint.


In [3]:
# # Load the dataset

from datasets import load_dataset

ds = load_dataset("mohammad-shirkhani/social_movielens_custom", split="train")
print(ds)
print("Rows:", len(ds))
print("Columns:", ds.column_names)

# Peek first row
ex0 = ds[0]
print("\nFirst row keys:", list(ex0.keys()))

# Pretty-short preview
for k, v in ex0.items():
    if isinstance(v, list) and len(v) > 3:
        print(f"- {k}: list(len={len(v)}), first 2:\n  {v[:2]}")
    else:
        print(f"- {k}: {v}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/691 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['user', 'item', 'answer', 'paths'],
    num_rows: 20000
})
Rows: 20000
Columns: ['user', 'item', 'answer', 'paths']

First row keys: ['user', 'item', 'answer', 'paths']
- user: {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}
- item: {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}
- answer: 4.0
- paths: list(len=20), first 2:
  ['user_question -> user_item_2 (rating=2) -> Item{MovieID 98, Title "Silence of the Lambs, The (1991)", Release Date 01-Jan-1991, Genres Drama, Thriller} -> item_user_4 (rating=4) -> User{UserID 498, Age 26, Gender Male, Occupation writer} -> usersim -> User{UserID 58, Age 27, Gender Male, Occupation programmer} -> user_item_3 (rating=3) -> item_question', 'user_question -> usersim -> User{UserID 716, Age 36, Gender Female, Occupation administrator} -> user_item_5 (rating=5) -> Item{MovieID 517, Title "Manhattan (1979)", Release Date 01-Jan-1979, Genres Comedy, D

In [4]:
from typing import Dict, Any

def detect_fields(example: Dict[str, Any]):
    user_keys = ["user", "User", "user_dict", "UserDict"]
    item_keys = ["item", "Item", "item_dict", "ItemDict"]
    rating_keys = ["answer", "rating", "label", "score"]
    paths_keys = ["paths", "meta_paths", "metapaths", "path_list", "path", "metaPaths"]

    def find_key(candidates, predicate=None):
        for k in candidates:
            if k in example and (predicate(example[k]) if predicate else True):
                return k
        for k, v in example.items():
            if predicate and predicate(v):
                return k
        return None

    user_key = find_key(user_keys, predicate=lambda v: isinstance(v, dict))
    item_key = find_key(item_keys, predicate=lambda v: isinstance(v, dict))
    rating_key = find_key(rating_keys, predicate=lambda v: isinstance(v, (float, int)))
    paths_key = find_key(paths_keys, predicate=lambda v: isinstance(v, list))

    if user_key is None:
        user_key = next((k for k, v in example.items() if isinstance(v, dict)), None)
    if item_key is None:
        cand = [k for k, v in example.items() if isinstance(v, dict) and k != user_key]
        item_key = cand[0] if cand else None
    if rating_key is None:
        cand = [k for k, v in example.items() if isinstance(v, (float, int))]
        rating_key = cand[0] if cand else None
    if paths_key is None:
        cand = [k for k, v in example.items() if isinstance(v, list)]
        paths_key = cand[0] if cand else None

    return user_key, item_key, rating_key, paths_key

user_key, item_key, rating_key, paths_key = detect_fields(ex0)
print("Detected keys ->",
      "user:", user_key,
      "| item:", item_key,
      "| rating:", rating_key,
      "| paths:", paths_key)

# Extract first example pieces for later
user0 = ex0[user_key]
item0 = ex0[item_key]
rating0 = ex0[rating_key]
paths0 = ex0[paths_key]
print("\nUser example:", user0)
print("Item example:", item0)
print("Rating example:", rating0)
print("Meta-paths:", len(paths0))

Detected keys -> user: user | item: item | rating: answer | paths: paths

User example: {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}
Item example: {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}
Rating example: 4.0
Meta-paths: 20


In [5]:
# Output must ONLY contain <reason>...</reason> then <answer>...</answer>.

from typing import List

def format_kv_block(title: str, d: Dict[str, Any]) -> str:
    lines = [f"{title}:"]
    for k, v in d.items():
        lines.append(f"- {k}: {v}")
    return "\n".join(lines)

def format_meta_paths(paths: List[str]) -> str:
    lines = ["Meta-path evidence (each path from this user to the target item):"]
    for p in paths:
        p = p.strip()
        if not p.startswith("- "):
            p = "- " + p
        lines.append(p)
    return "\n".join(lines)

def build_prompt_evidence_then_answer(
    user: Dict[str, Any],
    item: Dict[str, Any],
    rating: float,
    meta_paths: List[str],
) -> str:
    user_block = format_kv_block("User", user)
    item_block = format_kv_block("Item", item)
    observed_rating_str = str(rating)
    meta_block = format_meta_paths(meta_paths)

    instruction = (
        "Task: Extract evidence from the provided data and then conclude the numerical rating.\n\n"
        "You are given a heterogeneous bipartite graph setting (users and items). Edges include:\n"
        "- user→item rating interactions (e.g., user_item_k with an explicit rating),\n"
        "- usersim (user-user similarity), and\n"
        "- itemsim (item-item similarity).\n\n"
        "What to do:\n"
        "1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:\n"
        "   - the user attributes,\n"
        "   - the item attributes, and\n"
        "   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).\n"
        "   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).\n"
        "2) Then produce two XML blocks ONLY (no extra text):\n"
        "   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states\n"
        "      what information you obtain from the user/item/meta-paths and how those signals combine.\n"
        "      Avoid wording like “because the rating is X”. Instead, present evidence → inference, and end with a neutral\n"
        "      sentence such as: “Therefore, the rating equals {R}.”\n"
        "   b) <answer>{R}</answer> — Put the observed rating number {R} exactly as given below.\n\n"
        "STRICT FORMAT RULES:\n"
        "- Output ONLY these two blocks in this order, nothing else:\n"
        "<reason>\n"
        "...your evidence-first explanation here...\n"
        "</reason>\n"
        "<answer>{R}</answer>\n"
        "- Do not invent attributes, paths, or ratings not present in the input. If something is unknown, treat it as unknown.\n"
    )

    observed_block = f"Observed rating (user → item): {observed_rating_str}"
    prompt = (
        instruction
        + "\n=== INPUT ===\n"
        + user_block + "\n\n"
        + item_block + "\n\n"
        + observed_block + "\n\n"
        + meta_block + "\n"
        + "=== END INPUT ===\n"
    )
    return prompt.replace("{R}", observed_rating_str)

prompt0 = build_prompt_evidence_then_answer(user0, item0, rating0, paths0)
print("Prompt:\n")
print(prompt0[:])


Prompt:

Task: Extract evidence from the provided data and then conclude the numerical rating.

You are given a heterogeneous bipartite graph setting (users and items). Edges include:
- user→item rating interactions (e.g., user_item_k with an explicit rating),
- usersim (user-user similarity), and
- itemsim (item-item similarity).

What to do:
1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:
   - the user attributes,
   - the item attributes, and
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
2) Then produce two XML blocks ONLY (no extra text):
   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states
      what information you obtain from the user/item/meta-paths and how those signals combine.
      Avoid wording like “because the rating is X”. In

In [6]:
# # Single example run with Gemini 2.5 Flash-Lite (OpenAI style)

import re

def call_gemini(prompt: str, max_tokens: int = 2048):
    resp = client.chat.completions.create(
        model=MODEL_ID,
        reasoning_effort=REASONING_EFFORT,  # "low"
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1,
    )
    # 'content' is the text we need
    return resp.choices[0].message.content

def parse_reason_answer(text: str):
    reason = None
    answer = None
    m_reason = re.search(r"<reason>(.*?)</reason>", text, flags=re.DOTALL | re.IGNORECASE)
    if m_reason:
        reason = m_reason.group(1).strip()
    m_answer = re.search(r"<answer>\s*([0-9]+(?:\.[0-9]+)?)\s*</answer>", text, flags=re.DOTALL | re.IGNORECASE)
    if m_answer:
        answer = m_answer.group(1).strip()
    return reason, answer

print("Sending one test call...")
out_text = call_gemini(prompt0, max_tokens=2048)
print("\n===== MODEL RAW OUTPUT =====\n", out_text[:1500], "\n")
r0, a0 = parse_reason_answer(out_text)
print("Parsed reason OK?" , r0 is not None)
print("Parsed answer:", a0)


Sending one test call...

===== MODEL RAW OUTPUT =====
 <reason>
The target user (UserID 911) is a 37-year-old female writer. The target item (MovieID 193) is a Drama released in 1983.
Several meta-paths indicate strong positive interactions involving the target user and the target item, or items/users with similar attributes.
Specifically, one path shows a direct user-item interaction: user_question (UserID 911) -> user_item_5 (rating=5) -> Item{MovieID 191, Title "Amadeus (1984)", Genres Drama, Mystery} -> item_user_5 (rating=5) -> User{UserID 527, ...} -> usersim -> User{UserID 379, ...} -> user_item_4 (rating=4) -> item_question (MovieID 193). This path involves a high rating (5) to a Drama movie (Amadeus) and a high rating (4) via a user similarity path, connecting to the target item.
Another path shows: user_question (UserID 911) -> user_item_3 (rating=3) -> Item{MovieID 215, Title "Field of Dreams (1989)", Genres Drama} -> itemsim -> Item{MovieID 210, ...} -> item_user_3 (rating

In [7]:
# # Rate limiting helpers (15 RPM, 250k TPM)

import time
import math
from collections import deque

RPM_LIMIT = 15           # requests per minute
TPM_LIMIT = 250_000      # tokens per minute (input only, rough)
OUT_TOKENS_BUDGET = 2048  # what we asked via max_tokens

class RateLimiter:
    def __init__(self, rpm: int, tpm: int):
        self.rpm = rpm
        self.tpm = tpm
        self.req_times = deque()
        self.token_times = deque()  # (timestamp, tokens)

    @staticmethod
    def estimate_tokens(text: str) -> int:
        return max(1, math.ceil(len(text) / 4))

    def wait(self, prompt_text: str, out_tokens:int = OUT_TOKENS_BUDGET):
        now = time.time()
        while self.req_times and now - self.req_times[0] > 60:
            self.req_times.popleft()
        while self.token_times and now - self.token_times[0][0] > 60:
            self.token_times.popleft()

        in_tokens = self.estimate_tokens(prompt_text)
        # ensure RPM
        while len(self.req_times) >= self.rpm:
            sleep_s = 60 - (now - self.req_times[0])
            time.sleep(max(0.01, sleep_s))
            now = time.time()
            while self.req_times and now - self.req_times[0] > 60:
                self.req_times.popleft()

        # ensure TPM
        used_tpm = sum(t for (_, t) in self.token_times)
        while used_tpm + in_tokens + out_tokens > self.tpm:
            sleep_s = 60 - (now - self.token_times[0][0])
            time.sleep(max(0.01, sleep_s))
            now = time.time()
            while self.token_times and now - self.token_times[0][0] > 60:
                self.token_times.popleft()
            used_tpm = sum(t for (_, t) in self.token_times)

        # record this request
        self.req_times.append(time.time())
        self.token_times.append((time.time(), in_tokens + out_tokens))

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

def run_one_example(example, max_tokens:int=2048):
    user = example[user_key]
    item = example[item_key]
    rating = example[rating_key]
    meta = example[paths_key]

    prompt = build_prompt_evidence_then_answer(user, item, rating, meta)

    # Rate limit
    limiter.wait(prompt, out_tokens=max_tokens)

    # Retry with exponential backoff on transient errors
    backoff = 1.0
    for attempt in range(5):
        try:
            text = call_gemini(prompt, max_tokens=max_tokens)
            reason, answer = parse_reason_answer(text)
            return text, reason, answer
        except Exception as e:
            if attempt == 4:
                raise
            time.sleep(backoff)
            backoff *= 2.0


In [8]:
import math, re

# Config
INDEX = 0                
MAX_TOKENS_OUT = 2048    
SHOW_FULL_PROMPT = True   

# Ensure keys are detected
try:
    user_key, item_key, rating_key, paths_key
except NameError:
    ex_tmp = ds[0]
    user_key, item_key, rating_key, paths_key = detect_fields(ex_tmp)

# Select one example
ex = ds[INDEX]

# Build the exact prompt 
prompt = build_prompt_evidence_then_answer(
    ex[user_key],
    ex[item_key],
    ex[rating_key],
    ex[paths_key]
)

def est_tokens(s: str) -> int:
    return max(1, math.ceil(len(s) / 4))

print("=== PROMPT ===")
if SHOW_FULL_PROMPT:
    print(prompt)
else:
    preview_len = 1200
    print(prompt[:preview_len] + ("\n...\n" if len(prompt) > preview_len else ""))

print(f"\n[Prompt chars: {len(prompt):,} | est tokens: {est_tokens(prompt):,}]")

# Call the model
resp = client.chat.completions.create(
    model=MODEL_ID,                       
    reasoning_effort=REASONING_EFFORT,    
    messages=[{"role": "user", "content": prompt}],
    max_tokens=MAX_TOKENS_OUT,
    n=1,
)

text = resp.choices[0].message.content

# Show OUTPUT 
print("\n=== MODEL OUTPUT ===")
print(text)
print(f"\n[Output chars: {len(text):,} | est tokens: {est_tokens(text):,}]")

# Parse <reason> and <answer>
m_reason = re.search(r"<reason>(.*?)</reason>", text, flags=re.DOTALL | re.IGNORECASE)
m_answer = re.search(r"<answer>\s*([0-9]+(?:\.[0-9]+)?)\s*</answer>", text, flags=re.DOTALL | re.IGNORECASE)

print("\n=== PARSED FIELDS ===")
print("Reason found:", bool(m_reason))
print("Answer value:", m_answer.group(1).strip() if m_answer else None)


=== PROMPT ===
Task: Extract evidence from the provided data and then conclude the numerical rating.

You are given a heterogeneous bipartite graph setting (users and items). Edges include:
- user→item rating interactions (e.g., user_item_k with an explicit rating),
- usersim (user-user similarity), and
- itemsim (item-item similarity).

What to do:
1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:
   - the user attributes,
   - the item attributes, and
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
2) Then produce two XML blocks ONLY (no extra text):
   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states
      what information you obtain from the user/item/meta-paths and how those signals combine.
      Avoid wording like “because the rating is 

In [9]:
# # Run on 500 examples with tqdm and save JSONL

import json
from tqdm.auto import tqdm

MAX_EXAMPLES = min(500, len(ds))
OUT_PATH = "/content/gemini_flash_lite_reason_answer_500.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(MAX_EXAMPLES), desc="Generating explanations (Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        ex = ds[i]
        try:
            content, reason, answer = run_one_example(ex, max_tokens=2048)
            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone.")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Generating explanations (Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/500 [00:00<?, ?it/s]


Done.
OK: 494 | Missing reason: 5 | Missing answer: 5 | Failures: 1
Saved to: /content/gemini_flash_lite_reason_answer_500.jsonl

Sample outputs:
{'index': 0, 'user': {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}, 'item': {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}, 'rating': 4.0, 'reason': 'The user (UserID 911) is a 37-year-old female writer, and the target item (MovieID 193) is a Drama film. Several meta-paths provide insights into potential user preferences and item characteristics.\n\nOne path connects through UserID 716, a 36-year-old female administrator, who rated "Streetcar Named Desire, A (1951)" (Drama) a 5. This movie shares the Drama genre with the target item.\nAnother path shows UserID 498, a 26-year-old male writer, who interacted with "Silence of the Lambs, The (1991)" (Drama, Thriller) with a rating of 4, indicating a preference for Drama films among users with the same occupation.\nA 

In [10]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

# Rebuild the OpenAI-compatible Gemini client with the new key
client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

# Config: indices and output path
START_INDEX = 500
END_INDEX   = 1500   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_500_1499.jsonl"

# ---- Counters ----
ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 500..1499, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 500..1499).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 500..1499, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/1000…


Done (rows 500..1499).
OK: 937 | Missing reason: 15 | Missing answer: 15 | Failures: 48
Saved to: /content/gemini_flash_lite_reason_answer_500_1499.jsonl

Sample outputs:
{'index': 500, 'user': {'UserID': 398, 'Age': 40, 'Gender': 'Male', 'Occupation': 'other'}, 'item': {'MovieID': 602, 'Title': 'American in Paris, An (1951)', 'ReleaseDate': '01-Jan-1951', 'Genres': 'Musical, Romance'}, 'rating': 4.0, 'reason': 'The user (UserID 398) is male, 40 years old, and has an occupation categorized as \'other\'. The target item is the movie "An American in Paris" (MovieID 602), released in 1951, with genres Musical and Romance. The observed rating is 4.0.\n\nAnalyzing the meta-paths:\nSeveral paths connect User 398 to Item 602 and provide insights:\n1.  **Paths showing direct user-item interaction patterns with high ratings:**\n    *   `user_question -> user_item_5 (rating=5) -> Item{MovieID 485, Title "My Fair Lady (1964)", Genres Musical, Romance} -> ... -> item_question`: This path involves

In [11]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 1400
END_INDEX   = 2300   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_1400_2299.jsonl"

# Counters 
ok = 0
missing_reason = 0
missing_answer = 0
failures = 0


limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 1400..2299, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 1400..2299).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 1400..2299, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 1400..2299).
OK: 889 | Missing reason: 11 | Missing answer: 11 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_1400_2299.jsonl

Sample outputs:
{'index': 1400, 'user': {'UserID': 908, 'Age': 44, 'Gender': 'Female', 'Occupation': 'librarian'}, 'item': {'MovieID': 515, 'Title': 'Boot, Das (1981)', 'ReleaseDate': '04-Apr-1997', 'Genres': 'Action, Drama, War'}, 'rating': 4.0, 'reason': 'The target item, "Boot, Das (1981)", is classified under the genres Action, Drama, and War. The user, UserID 908, is a 44-year-old female librarian.\n\nSeveral meta-paths provide evidence:\n1.  A path connects User 908 to Item 515 through Item 318 ("Schindler\'s List (1993)"), which shares the Drama and War genres with the target item. This path indicates a `user_item_5` (rating=5) interaction with Item 318, followed by an `item_user_4` (rating=4) interaction with User 269 (a librarian, like User 908), who is similar (`usersim`) to User 18, who then had a `user_item_5` (rating=5

In [12]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 2300
END_INDEX   = 3200   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_2300_3199.jsonl"

# Counters
ok = 0
missing_reason = 0
missing_answer = 0
failures = 0


limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 2300..3199, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 2300..3199).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 2300..3199, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 2300..3199).
OK: 884 | Missing reason: 14 | Missing answer: 16 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_2300_3199.jsonl

Sample outputs:
{'index': 2300, 'user': {'UserID': 690, 'Age': 35, 'Gender': 'Male', 'Occupation': 'salesman'}, 'item': {'MovieID': 1041, 'Title': 'Forget Paris (1995)', 'ReleaseDate': '01-Jan-1995', 'Genres': 'Comedy, Romance'}, 'rating': 3.0, 'reason': 'The target item "Forget Paris (1995)" is a Comedy and Romance movie.\nThe user, UserID 690, is a 35-year-old Male salesman.\nSeveral meta-paths indicate User 690\'s preferences and connections.\nOne path shows User 690 interacted with "Four Weddings and a Funeral (1994)", which shares the genres Comedy and Romance with the target item, but with a low rating of 2. Another path also shows User 690 interacting with "Four Weddings and a Funeral (1994)" with a rating of 2.\nHowever, User 690 also interacted with "Stand by Me (1986)" (Adventure, Comedy, Drama) with a rating of 4, and "A

In [13]:

import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 3200
END_INDEX   = 4100   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_3200_4099.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 3200..4099, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 3200..4099).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 3200..4099, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 3200..4099).
OK: 882 | Missing reason: 17 | Missing answer: 18 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_3200_4099.jsonl

Sample outputs:
{'index': 3200, 'user': {'UserID': 436, 'Age': 30, 'Gender': 'Female', 'Occupation': 'administrator'}, 'item': {'MovieID': 200, 'Title': 'Shining, The (1980)', 'ReleaseDate': '01-Jan-1980', 'Genres': 'Horror'}, 'rating': 3.0, 'reason': 'The user, UserID 436, is a 30-year-old female administrator. The target item, MovieID 200, is "Shining, The (1980)" with the genre Horror.\n\nWe analyze the meta-paths for signals related to the user\'s preferences and the item\'s genre:\n\n1.  Several paths indicate the user has interacted with Horror genre movies:\n    *   A path connects through Item{MovieID 447, Genres Horror} with a user_item rating of 1.\n    *   Another path connects through Item{MovieID 635, Genres Horror} with a user_item rating of 3.\n    *   A path connects through Item{MovieID 928, Genres Drama, Horror} w

In [14]:

import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 4100
END_INDEX   = 5000   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_4100_4999.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0


limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 4100..4999, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 4100..4999).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 4100..4999, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 4100..4999).
OK: 878 | Missing reason: 21 | Missing answer: 22 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_4100_4999.jsonl

Sample outputs:
{'index': 4100, 'user': {'UserID': 405, 'Age': 22, 'Gender': 'Female', 'Occupation': 'healthcare'}, 'item': {'MovieID': 1268, 'Title': 'Bitter Moon (1992)', 'ReleaseDate': '01-Jan-1992', 'Genres': 'Drama'}, 'rating': 1.0, 'reason': 'The target user (UserID 405) is Female, aged 22, and works in healthcare. The target item (MovieID 1268) is a Drama from 1992.\n\nThe meta-paths provide several signals:\n- Path 10 shows User 405 interacting with Item 462 ("Like Water For Chocolate") which is listed as Drama, Romance, and received a rating of 5 from the user (via `user_item_2`). This suggests a preference for Drama and Romance genres.\n- Path 13 shows User 405 interacting with Item 736 ("Shadowlands") which is listed as Drama, Romance, and received a rating of 5 from the user (via `user_item_5`). This further supports a 

In [15]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 5000
END_INDEX   = 5900   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_5000_5899.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 5000..5899, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 5000..5899).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 5000..5899, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 5000..5899).
OK: 887 | Missing reason: 13 | Missing answer: 13 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_5000_5899.jsonl

Sample outputs:
{'index': 5000, 'user': {'UserID': 210, 'Age': 39, 'Gender': 'Male', 'Occupation': 'engineer'}, 'item': {'MovieID': 482, 'Title': 'Some Like It Hot (1959)', 'ReleaseDate': '01-Jan-1959', 'Genres': 'Comedy, Crime'}, 'rating': 5.0, 'reason': 'The user (UserID 210) is a 39-year-old male engineer. The target item (MovieID 482) is "Some Like It Hot (1959)", a Comedy and Crime film. The observed rating is 5.0.\n\nMeta-path analysis reveals several corroborating signals:\n1.  User 210 exhibits a preference for Comedy films through multiple paths. For instance, a path shows User 210 indirectly liking Item 523 ("Cool Hand Luke", Comedy, Drama) with a rating of 4. Item 523 is linked via `itemsim` to other items, and through a chain involving Item 430 ("Duck Soup", Comedy, War), it eventually leads to the target item. Another 